In [3]:
import sys
import numpy as np
from keras.layers import Input,Dense,Reshape,Flatten,Dropout,BatchNormalization,Lambda,concatenate
from keras.layers.core import Activation
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers.convolutional import Convolution2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential,Model
from keras.optimizers import Adam,SGD,nadam,Adamax
import keras.backend as k
from keras.utils import plot_model

ModuleNotFoundError: No module named 'keras_contrib'

In [4]:
class Discriminator(object):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-4-c3d9d66e3eb6>, line 2)

In [7]:
# from gan import GAN
# from generator import Generator
# from discriminator import Discriminator
from keras.layers import Input
from random import randint
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import os
from PIL import Image

In [9]:
class Trainer:
    def __init__(self,height=64,width=64,epochs=50000,batch=32,checkpoint=50,
                 train_data_path_A='',train_data_path_B='',
                 test_data_path_A='',test_data_path_A=''):
        self.EPOCHS =  epochs
        self.BATCH = batch
        self.RESIZE_HEIGHT = height
        self.RESIZE_WIDTH = width
        self.CHECKPOINT = checkpoint

        # train셋과 test셋 A,B 두개(총 4개)
        self.X_train_A,self.H_A,self.W_A,self.C_A = self.load_data(train_data_path_A)
        self.X_train_B,self.H_B,self.W_B,self.C_B = self.load_data(train_data_path_B)

        self.X_train_A,self.H_A,self.W_A,self.C_A = self.load_data(train_data_path_A)
        self.X_train_B,self.H_B,self.W_B,self.C_B = self.load_data(train_data_path_B)

        # generator A2B,B2A로 2개
        self.generator_A_to_B = Generator(height=self.H_A,width=self.W_A,channels=self.C_A)
        self.generator_B_to_A = Generator(height=self.H_B,width=self.W_B,channels=self.C_B)


        # 내용 추가
        self.orig_A = Input(shape=(self.W_A,self.H_A,self.C_A))
        self.orig_B = Input(shape=(self.W_B,self.H_B,self.C_B))

        # fake_A, fake_B(원래 -> 가짜)
        self.fake_B = self.generator_A_to_B.Generator(self.orig_A)
        self.fake_A = self.generator_B_to_A.Generator(self.orig_B)

        # reconstruced(가짜 -> 원래)
        self.reconstructed_A = self.generator_B_to_A.Generator(self.fake_B)
        self.reconstructed_B = self.generator_A_to_B.Generator(self.fake_A)

        self.id_A = self.generator_B_to_A.Generator(self.orig_A)
        self.id_B = self.generator_A_to_B.Generator(self.orig_B)

        # 판별기
        self.discriminator_A = Discriminator(height=self.H_A,width=self.W_A,channels=self.C_A)
        self.discriminator_B = Discriminator(height=self.H_B,width=self.W_B,channels=self.C_B)
        # 판별기 훈련 x
        self.discriminator_A.trainable = False
        self.discriminator_B.trainable = False
        self.valid_A = self.discriminator_A.Discriminator(self.fake_A)
        self.valid_B = self.discriminator_B.Discriminator(self.fake_B)

        model_inputs = [self.orig_A,self.orig_B]
        model_outputs = [self.valid_A,self.valid_B,
                         self.reconstructed_A,self.reconstructed_B,
                         self.id_A,self.id_B]
        self.gan = GAN(model_inputs=model_inputs,
                       model_outputs=model_outputs,
                       lambda_cycle=10.0,
                       lambda_id=1.0)
        
        def train(self):
            for e in range(self.EPOCHS):
                b = 0
                X_train_A_temp = deepcopy(self.X_train_A)
                X_train_B_temp = deepcopy(self.X_train_B)
                while min(len(X_train_A_temp),len(X_train_B_temp)) > self.BATCH:
                    b = b + 1
                    # A,B에서 진짜 이미지 선택
                    count_real_images = int(self.BATCH)
                    starting_indexs = randint(0,(min(len(X_train_A_temp),len(X_train_B_temp)) - count_real_images))
                    real_images_raw_A = X_train_A_temp[starting_indexs:(starting_indexs + count_real_images)]
                    real_images_raw_B = X_train_B_temp[starting_indexs:(starting_indexs + count_real_images)]

                    # 선택된 이미지들 지워줌
                    X_train_A_temp = np.delete(X_train_A_temp.range(starting_indexs,(starting_indexs + count_real_images)),0)
                    X_train_B_temp = np.delete(X_train_B_temp.range(starting_indexs,(starting_indexs + count_real_images)),0)

                    batch_A = real_images_raw_A.reshape(count_real_images,self.W_A,self.H_A,self.C_A)                    
                    batch_A = real_images_raw_A.reshape(count_real_images,self.W_A,self.H_A,self.C_A)

                    if self.flipCoin():
                        x_batch_A = batch_A
                        x_batch_B = batch_B
                        y_batch_A = np.ones([count_real_images,1])
                        y_batch_B = np.ones([count_real_images,1])
                    else:
                        x_batch_A = self.generator_B_to_A.Generator.predict(batch_B)
                        x_batch_B = self.generator_A_to_B.Generator.predict(batch_A)
                        y_batch_A = np.zeros([self.BATCH,1])
                        y_batch_B = np.zeros([self.BATCH,1])
                    
                    # 판별기 A와 판별기 B를 작성한다.
                    self.discriminator_A.Discriminator.trainable = True
                    discriminator_loss_A = self.discriminator_A.Discriminator.train_on_batch(x_batch_A,y_batch_A)[0]
                    self.discriminator_A.Discriminator.trainable = False

                    self.discriminator_B.Discriminator.trainable = True
                    discriminator_loss_B = self.discriminator_B.Discriminator.train_on_batch(x_batch_B,y_batch_B)[0]
                    self.discriminator_B.Discriminator.trainable = False



                    # 점검
                    print (f'Batch: {str(int(b))}, [Full Discriminator :: Loss: {str(full_loss)}], [ Generator :: Loss: {str(generator_loss)}]')
                    if b % self.CHECKPOINT == 0 :
                        label = str(e)+'_'+str(b)
                        self.plot_checkpoint(label)

                print (f'Epoch: {str(int(e))}, [Full Discriminator :: Loss: {str(full_loss)}], [ Generator :: Loss: {str(generator_loss)}]'')




        # 기타 함수
        def load_data(self,data_path,amount_of_data = 1.0):
            listOFFiles = self.grabListOfFiles(data_path,extension="jpg")
            X_train = np.array(self.grabArrayOfImages(listOFFiles))
            height, width, channels = np.shape(X_train[0])
            X_train = X_train[:int(amount_of_data*float(len(X_train)))]
            X_train = (X_train.astype(np.float32) - 127.5)/127.5
            X_train = np.expand_dims(X_train, axis=3)
            return X_train, height, width, channels

        # 파일 목록 찾기
        def grabListOfFiles(self,startingDirectory,extension='.webp'):
            listOfFiles = []
            for file in os.listdir(startingDirectory):
                if file.endswith(extension):
                    listOfFiles.append(os.path.join(startingDirectory,file))
            return listOfFiles

        # 동전 던지기
        def flipCoin(self,chance=0.5):
            return np.random.binomial(1,chance)

        
        # 이미지 가져오기
        def grabArrayOfImages(self,listOfFiles,gray=False):
            imageArr = []
            for f in listOfFiles:
                if gray:
                    im = Image.open(f).convert("L")
                else:
                    im = Image.open(f).convert("RGB")
                imData = np.asarray(im)
                imageArr.append(imData)
            return imageArr









        


SyntaxError: duplicate argument 'test_data_path_A' in function definition (<ipython-input-9-01075ad87791>, line 5)